### **Quantum Entanglement**
**Quantum entanglement** links two particles so that measuring one instantly tells us about the other — even if they're far apart. Einstein called this “spooky action at a distance.” In 1964, **Bell’s inequality** was proposed to test if particles follow classical rules. The **CHSH experiment** showed that entangled particles behave in ways classical physics can’t explain. In 1997, scientists used entanglement to achieve **quantum teleportation** — sending a quantum state without moving the particle itself.

Let’s use **Qiskit** to explore the CHSH test and quantum teleportation!

### **CHSH Game**
The CHSH game is a thought experiment based on ideas from quantum physics. It was proposed as a way to test whether the world behaves according to classical rules or quantum mechanics. In 1964, physicist [John Bell](https://en.wikipedia.org/wiki/John_Stewart_Bell) introduced Bell's inequality to show that certain predictions of quantum mechanics can't be explained by any local classical theory. Later, Clauser, Horne, Shimony, and Holt (CHSH) created a practical version of this test — now called the CHSH game. The game involves two players, Alice and Bob, who cannot talk to each other once the game starts. They try to win a challenge given by a Referee by using a strategy they agree on beforehand.

Here’s how the CHSH game works:

1. The Referee randomly picks two input bits: `x` and `y`.
2. `x` is sent to Alice, and `y` is sent to Bob.
3. Without knowing each other's input, Alice and Bob each send back one bit: `a` and `b`.
4. **They win if:** `a XOR b == x AND y`

This means:
- In most cases, when `x` and `y` are not both 1, Alice and Bob should give the same answer.
- If both `x = 1` and `y = 1`, they must give different answers.

Since they can't communicate, they need a strategy that works on average. With only classical logic, the best they can do is win **75%** of the time.


### **Classical Limit**
The classical limit of this game is win_probability $\leq = 75% $. To understand why the classical success rate in the CHSH game is limited to 75%, let’s look at all possible input combinations the Referee might send:
$$(x, y) \in \{(0,0), (0,1), (1,0), (1,1)\}$$
The win condition is:
$$a XOR b == x AND y$$
This means:
- For (0,0), (0,1), and (1,0), the right side is 0 → Alice and Bob must give the **same** output.
- For (1,1), the right side is 1 → they must give **different** outputs.

Now suppose Alice and Bob agree on a fixed classical strategy, like always outputting 0.  
They will win for the first 3 cases, but lose when (x, y) = (1,1), because:
$$0 XOR 0 = 0 \neq 1 $$
No matter what fixed classical strategy they use, **only 3 out of 4 cases can be satisfied**. This means:

**The best possible classical success rate is 75%.**

In [ ]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.visualization import plot_histogram, plot_distribution
from qiskit.circuit import Parameter
from qiskit_ibm_runtime import Options, Session, SamplerV2 as Sampler, QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit.result import marginal_distribution
from qiskit.transpiler import generate_preset_pass_manager
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import io